# Getting NUS Modules After a Specific Time

Sometimes, we don't know what modules to take on a semester but you know you only have certain timeslots to fill. This code will give you a list of modules that might answer your queries!

## Breakdown

First of all, import requests because we are to extract data from the NUSMods API.

In [1]:
import requests

Next up, set the AY, sem, and cutoff time.

Also define a variable wanted_mods to store our final result.

The full data URL is put below in case you want to go deeper into the JSON structure.

In [2]:
AY = '2021-2022'
sem = 1
cutoff_time = 1800    # I want modules that have class at 6 PM or later
wanted_mods = []

print('https://api.nusmods.com/v2/' + AY + '/moduleList.json')

https://api.nusmods.com/v2/2021-2022/moduleList.json


Using requests, we obtain the JSON of the data and convert it into a list of dictionaries.

In [3]:
module_list = requests.get('https://api.nusmods.com/v2/' + AY + '/moduleList.json').json()

module_list is now a list of dictionaries, each dictionary has a structure just like below.

In [4]:
module_list[949] # Yes, I set the index on purpose

{'moduleCode': 'CS2040',
 'semesters': [1, 2],
 'title': 'Data Structures and Algorithms'}

Now we iterate for every dictionary in module_list as follows.


*   The default template checks for modules only below 3000. You can add more conditionals if you want to.
*   For every module code, there is another link where we can extract the data from.
*   If there is a timetable that permits, add the module to our final result.

Before we start iterating, let's see how the data of a single module will look like.



### Single Module Data

In [5]:
cs1010s_data = requests.get('https://api.nusmods.com/v2/' + AY + '/modules/CS1010S.json').json()
list(cs1010s_data.keys()) # the main tree

['acadYear',
 'preclusion',
 'description',
 'title',
 'department',
 'faculty',
 'workload',
 'moduleCredit',
 'moduleCode',
 'attributes',
 'aliases',
 'semesterData',
 'fulfillRequirements']

In [6]:
list(cs1010s_data['semesterData'][0].keys()) # pick one branch from the main tree, it has sub-branches as shown

['semester', 'timetable', 'covidZones', 'examDate', 'examDuration']

In [7]:
cs1010s_data['semesterData'][0]['timetable'][0] # pick one sub-branch from it, turns out it has many leaves, pick the first leaf

{'classNo': '15',
 'covidZone': 'Unknown',
 'day': 'Friday',
 'endTime': '1500',
 'lessonType': 'Recitation',
 'size': 50,
 'startTime': '1400',
 'venue': 'E-Learn_C',
 'weeks': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]}

In [8]:
print('https://api.nusmods.com/v2/' + AY + '/modules/CS1010S.json') # view full data here

https://api.nusmods.com/v2/2021-2022/modules/CS1010S.json


### Main Code

In [9]:
# This might take a while...

for mod in module_list:
    mod_code = mod['moduleCode']

    mod_num = ''.join(filter(lambda x: x.isdigit(), mod_code))

    if int(mod_num) < 3000:
        mod_info = requests.get('https://api.nusmods.com/v2/' + AY + '/modules/' + mod_code + '.json').json()

        for mod_info_sem in mod_info['semesterData']:             # mod_info['semesterData'] is a list of dictionaries too!
            if mod_info_sem['semester'] == sem:
                for lesson in mod_info_sem['timetable']:          # also a list of dictionaries, structure shown above
                    if int(lesson['startTime']) >= cutoff_time:   # you want the module to start at cutoff_time or later
                        wanted_mods.append(mod_code)
                        # print(mod_code)                         # track progress so far, optional
                        break                                     # this assumes that if there is at least one class (no matter what type)
                                                                  # that satisfies the time constraint, add the module

                                                                  # feel free to add another constraint(s), e.g. it must be a lecture,
                                                                  # hence put lesson['lessonType'] == 'Lecture'
                                                                  # or maybe specify the prefix of the module code, e.g.
                                                                  # put mod_code[:2] = 'MA'
                                                                  # another one idea is to use time intervals, e.g.
                                                                  # int(lesson['startTime']) >= lower_bound and int(lesson['endtTime']) <= upper_bound

## Final output is as shown.

In [10]:
print(wanted_mods)

['BPM1701', 'BPM1702', 'BPM1705', 'CN2121E', 'CN2122E', 'CS1010S', 'CS1101S', 'DAO1704', 'DMB1201DAO', 'DMR1201GEQA', 'DTK1234', 'EG2101', 'EG2201A', 'EG2301', 'EG2311', 'EG2401A', 'ES1531', 'ES2531', 'FSC2101', 'GEC1024', 'GEC1030', 'GEC1031', 'GEH1009', 'GEH1036', 'GEH1062', 'GEH1077', 'GEH1079', 'GEK1505', 'GES1000T', 'GES1002T', 'GES1007', 'GES1028', 'GES1035', 'GESS1005', 'GESS1020', 'GESS1025', 'GET1008', 'GET1021', 'GET1029', 'GET1042', 'GEX1000', 'GEX1005', 'GEX1015', 'HS1501', 'HY1101E', 'HY2253', 'IE2010E', 'IE2110E', 'IE2130E', 'LAB1201', 'LAB2201', 'LAC1201', 'LAF1201', 'LAG1201', 'LAG2201', 'LAH1201', 'LAJ1201', 'LAJ2201', 'LAJ2202', 'LAJ2203', 'LAK1201', 'LAK2201', 'LAL2201', 'LAM1201', 'LAR1201', 'LAS1201', 'LAT1201', 'LAV1201', 'LC1016', 'LC1025', 'MA1521', 'ME2121E', 'ME2134E', 'ME2142E', 'ME2151E', 'MKT2711A', 'MNO1706B', 'MUA1107', 'MUA1108', 'MUA2107', 'MUA2108', 'NM2103', 'NM2104', 'NM2201', 'NM2207', 'NM2220', 'NM2302', 'NUR1123', 'NUR2500', 'NUR2501', 'PL1101E', 

## Putting it all together

In [13]:
def get_mods_after_time(AY, sem, cutoff_time):
  module_list = requests.get('https://api.nusmods.com/v2/' + AY + '/moduleList.json').json()

  wanted_mods = []

  for mod in module_list:
    mod_code = mod['moduleCode']

    mod_num = ''.join(filter(lambda x: x.isdigit(), mod_code))

    if int(mod_num) < 3000:
        mod_info = requests.get('https://api.nusmods.com/v2/' + AY + '/modules/' + mod_code + '.json').json()

        for mod_info_sem in mod_info['semesterData']:
            if mod_info_sem['semester'] == sem:
                for lesson in mod_info_sem['timetable']:
                    if int(lesson['startTime']) >= cutoff_time:
                        wanted_mods.append(mod_code)
                        break
  
  return wanted_mods

# The whole explanation above is basically about running get_mods_after_time('2021-2022', 1, 1800)

## Appendix

In [12]:
def get_module_info(start_year, mod_code):
  AY = f'{start_year}-{start_year+1}'

  print('https://api.nusmods.com/v2/' + AY + '/modules/' + mod_code + '.json')
  return requests.get('https://api.nusmods.com/v2/' + AY + '/modules/' + mod_code + '.json').json()

cs2040_data = get_module_info(2021, 'CS2040')
print()
dict(filter(lambda x: x[0] != 'semesterData', cs2040_data.items()))

https://api.nusmods.com/v2/2021-2022/modules/CS2040.json



{'acadYear': '2021/2022',
 'aliases': ['CS2040C', 'CS2040S'],
 'attributes': {'mpes1': True, 'mpes2': True},
 'department': 'Computer Science',
 'description': 'This module introduces students to the design and implementation of fundamental data structures and algorithms. The module covers basic data structures (linked lists, stacks, queues, hash tables, binary heaps, trees, and graphs), searching and sorting algorithms, and basic analysis of algorithms.',
 'faculty': 'Computing',
 'fulfillRequirements': ['CS2102',
  'CS2103',
  'CS2105',
  'CS2220',
  'CS2309',
  'CS3223',
  'CS3225',
  'CS3230',
  'CS3241',
  'CS3243',
  'CS3244',
  'CS4236',
  'CS4243',
  'CS5240',
  'CS3245',
  'CS4215',
  'CS5343',
  'CS2103T',
  'CS3218',
  'CS2108',
  'CS2113',
  'CS2113T',
  'CS5332',
  'CS5346',
  'CP3107',
  'CS4261',
  'CS5477',
  'CS5461',
  'CS4277',
  'CS5469',
  'CS4269',
  'CS3231',
  'IS3261',
  'BT4015',
  'CG2271'],
 'moduleCode': 'CS2040',
 'moduleCredit': '4',
 'preclusion': 'CS102